# 한국판 케빈 베이컨 놀이
$$ $$
> 케빈 베이컨의 여섯 다리(영어: Six Degrees of Kevin Bacon)는 어떤 배우와 케빈 베이컨까지 최단의 연결 고리를 만드는 놀이이다.  
이는 케빈 케이컨이 매우 다작(多作)하는 배우라는 것이 근거가 된다.
* 출처: [[위키백과] 케빈 베이컨의 여섯 다리](https://ko.wikipedia.org/wiki/%EC%BC%80%EB%B9%88_%EB%B2%A0%EC%9D%B4%EC%BB%A8%EC%9D%98_%EC%97%AC%EC%84%AF_%EB%8B%A4%EB%A6%AC)

$$ $$
> 한국영화에 출연한 한국 영화배우들을 대상으로 "케빈 베이컨 놀이"를 직접 해보았습니다.  
* 기준일자: 2017년 11월 23일

In [1]:
# 경고문구를 출력하지 않도록 설정
options(warn = -1)

In [2]:
# 라이브러리 불러오기
library(readxl)
library(reshape2)
library(igraph)
library(network)
library(sna)
library(ggplot2)
library(GGally)
library(scales)
library(Hmisc)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

network: Classes for Relational Data
Version 1.13.0 created on 2015-08-31.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


Attaching package: ‘network’

The following objects are masked from ‘package:igraph’:

    %c%, %s%, add.edges, add.vertices, delete.edges, delete.vertices,
    get.edge.attribute, get.edges, get.vertex.attribute, is.bipartite,
    is.directed, list.edge.attributes, list.vertex.attributes,
    set.edge.attribute, se

In [3]:
# 작업폴더 경로 확인 
getwd()

# 데이터가 저장된 폴더로 작업폴더 변경
setwd("./data")

[1] "/Users/drkevin/Documents/GitHub/DrKevin22/SixDegreeKorVer"

In [4]:
# 작업폴더 내 저장된 xlsx 파일 확인
list.files(pattern = "xlsx")

[1] "Korean movie actors_20170714.xlsx" "Korean movie actors_20171123.xlsx"
[3] "Korean movie year_20170714.xlsx"   "Korean movie year_20171123.xlsx"  
[5] "Korean movie_20170714.xlsx"        "Korean movie_20171123.xlsx"

In [5]:
# 한국영화 리스트 불러오기 
movieList <- read_excel("Korean movie_20171123.xlsx")

# 데이터 구조 확인
str(movieList)

# 첫 10줄만 미리보기
head(movieList, 10L)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	26894 obs. of  3 variables:
 $ movieCode: chr  "68831" "46851" "48065" "99481" ...
 $ movieLink: chr  "http://movie.naver.com/movie/bi/mi/basic.nhn?code=68831" "http://movie.naver.com/movie/bi/mi/basic.nhn?code=46851" "http://movie.naver.com/movie/bi/mi/basic.nhn?code=48065" "http://movie.naver.com/movie/bi/mi/basic.nhn?code=99481" ...
 $ movieName: chr  "0.5" "0.6˚의 재앙 (The Disaster 0.6˚)" "0.7" "0.7%의 삶" ...


movieCode,movieLink,movieName
68831,http://movie.naver.com/movie/bi/mi/basic.nhn?code=68831,0.5
46851,http://movie.naver.com/movie/bi/mi/basic.nhn?code=46851,0.6˚의 재앙 (The Disaster 0.6˚)
48065,http://movie.naver.com/movie/bi/mi/basic.nhn?code=48065,0.7
99481,http://movie.naver.com/movie/bi/mi/basic.nhn?code=99481,0.7%의 삶
147380,http://movie.naver.com/movie/bi/mi/basic.nhn?code=147380,0000
116885,http://movie.naver.com/movie/bi/mi/basic.nhn?code=116885,007 수퍼맨의 귀향 (The Return of 007 Superman)
41338,http://movie.naver.com/movie/bi/mi/basic.nhn?code=41338,007 폭소 대작전
44587,http://movie.naver.com/movie/bi/mi/basic.nhn?code=44587,00씨의 하루 (A Day With Mr. 00)
29688,http://movie.naver.com/movie/bi/mi/basic.nhn?code=29688,01412 파사신검 (01412 破邪神劍)
106269,http://movie.naver.com/movie/bi/mi/basic.nhn?code=106269,"0609_청계산 (A Weekend, and the Mountain)"


In [6]:
# 한국영화 제작년도 데이터 불러오기
myearList <- read_excel("Korean movie year_20171123.xlsx")

# 데이터 구조 확인
str(myearList)

# 첫 10줄만 미리보기
head(myearList, 10L)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	25571 obs. of  2 variables:
 $ movieCode: chr  "68831" "46851" "48065" "99481" ...
 $ year     : chr  "2006" "2005" "2001" "2012" ...


movieCode,year
68831,2006
46851,2005
48065,2001
99481,2012
147380,2015
116885,2007
41338,1991
44587,2007
106269,2012
130258,2014


In [7]:
# 한국영화 출연배우 리스트 불러오기
actorList <- read_excel("Korean movie actors_20171123.xlsx")

# 데이터 구조 확인
str(actorList)

# 첫 10줄만 미리보기
head(actorList, 10L)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	109234 obs. of  4 variables:
 $ actorCode: chr  "381876" "363129" "406889" "389265" ...
 $ actorLink: chr  "http://movie.naver.com/movie/bi/pi/basic.nhn?code=381876" "http://movie.naver.com/movie/bi/pi/basic.nhn?code=363129" "http://movie.naver.com/movie/bi/pi/basic.nhn?code=406889" "http://movie.naver.com/movie/bi/pi/basic.nhn?code=389265" ...
 $ actorName: chr  "박현수" "이보리" "전혜원" "이민지" ...
 $ movieCode: chr  "147380" "147380" "147380" "147380" ...


actorCode,actorLink,actorName,movieCode
381876,http://movie.naver.com/movie/bi/pi/basic.nhn?code=381876,박현수,147380
363129,http://movie.naver.com/movie/bi/pi/basic.nhn?code=363129,이보리,147380
406889,http://movie.naver.com/movie/bi/pi/basic.nhn?code=406889,전혜원,147380
389265,http://movie.naver.com/movie/bi/pi/basic.nhn?code=389265,이민지,147380
389266,http://movie.naver.com/movie/bi/pi/basic.nhn?code=389266,공예림,147380
1841,http://movie.naver.com/movie/bi/pi/basic.nhn?code=1841,심형래,41338
16803,http://movie.naver.com/movie/bi/pi/basic.nhn?code=16803,김학래,41338
18596,http://movie.naver.com/movie/bi/pi/basic.nhn?code=18596,박승대,41338
55490,http://movie.naver.com/movie/bi/pi/basic.nhn?code=55490,최선미,41338
67669,http://movie.naver.com/movie/bi/pi/basic.nhn?code=67669,곽재문,41338


In [8]:
# 불필요한 컬럼 삭제 (링크 컬럼)
movieList <- movieList[, c(-2)]
actorList <- actorList[, c(-2)]

# 데이터 프레임 합치기
df <- merge(x = actorList, 
            y = movieList, 
            by = "movieCode", 
            all.x = TRUE)

df <- merge(x = df, 
            y = myearList, 
            by = "movieCode", 
            all.x = TRUE)

# 데이터 프레임 구조 확인하기
str(df)

'data.frame':	109234 obs. of  5 variables:
 $ movieCode: chr  "100001" "100001" "100001" "100001" ...
 $ actorCode: chr  "40569" "48841" "97542" "42817" ...
 $ actorName: chr  "최승준" "시영준" "신용우" "윤여진" ...
 $ movieName: chr  "와라! 편의점" "와라! 편의점" "와라! 편의점" "와라! 편의점" ...
 $ year     : chr  "2009" "2009" "2009" "2009" ...


In [9]:
# 컬럼 순서 변경
df <- df[, c("movieCode","movieName","actorCode","actorName","year")]

# 영화코드 기준으로 오름차순 정렬
df <- df[order(df$movieCode),]

# 첫 10줄만 미리보기
head(df, 10)

movieCode,movieName,actorCode,actorName,year
100001,와라! 편의점,40569,최승준,2009
100001,와라! 편의점,48841,시영준,2009
100001,와라! 편의점,97542,신용우,2009
100001,와라! 편의점,42817,윤여진,2009
100001,와라! 편의점,5527,이계윤,2009
100009,박제된 공주 (Briar Rose),47790,정보름,2012
100009,박제된 공주 (Briar Rose),1712,김꽃지,2012
100009,박제된 공주 (Briar Rose),80534,이명진,2012
100017,검지손가락,5770,류덕환,2011
100017,검지손가락,48937,소이,2011


In [10]:
# 배우별 출연한 영화수 컬럼 만들기
numMovie <- aggregate(x = df$movieCode, 
                      by = list(df$actorCode, df$actorName), 
                      FUN = length)

# 컬럼명 바꾸기
colnames(numMovie) <- c("actorCode","actorName","movieCnt")

# 첫 10줄만 미리보기
head(numMovie, 10L)

actorCode,actorName,movieCnt
165414,2AM,3
174333,2PM,1
307425,45rpm,1
380209,가나메 유키코,1
6430,가네코 타카토시,1
378601,가다니엘,1
165479,가득희,12
303918,가람,1
307428,가리온,1
147878,가브리앨라 무스카와,1


In [11]:
# 배우-영화 데이터에 병합
df <- merge(x = df, 
            y = numMovie[,c(1,3)], 
            by = "actorCode", 
            all.x = TRUE)

# 배우별 출연한 영화수 빈도 확인
# "useNA" 인자를 추가하면 NA인 값의 빈도수도 함께 출력 
table(df$movieCnt, useNA = "ifany")


    1     2     3     4     5     6     7     8     9    10    11    12    13 
16085  6688  4806  3880  3210  2892  2520  2264  2205  1990  1881  1932  1599 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
 1848  1455  1776  1615  1494  1596  1600  1323  1276  1380  1200  1350   832 
   27    28    29    30    31    32    33    34    35    36    37    38    39 
 1215  1092  1334  1110  1333  1312  1023   714   630   936   555   912   702 
   40    41    42    43    44    45    46    47    48    49    50    51    52 
  440   656   588   731   748   720   506   658   576   588   600   255   572 
   53    54    55    56    57    58    59    60    61    63    64    65    66 
  318   540   165   168   570   232   236   360   305   378   256   520    66 
   67    68    69    70    71    72    73    74    75    76    77    78    79 
  201   340   345   210   142   216   365    74   150   304   154   234   158 
   80    81    82    83    84    86    87    88    

In [12]:
# NA 삭제
df <- df[is.na(df$movieCnt) == FALSE,]

In [13]:
# 가장 많은 영화를 찍은 배우의 출연작 리스트
df[df$movieCnt == max(df$movieCnt),]

# 우리나라에서 가장 많은 영화를 찍은 배우는 총 505편의 영화를 찍은 '강신성일'

,actorCode,movieCode,movieName,actorName,year,movieCnt
20806,1833,21884,아무리 미워도,강신성일,1969,505
20807,1833,21904,아스팔트 위의 여자,강신성일,1978,505
20808,1833,19772,고백,강신성일,1971,505
20809,1833,23201,청춘 교실 (청춘교실),강신성일,1963,505
20810,1833,19897,그 얼굴에 햇살을 (Sunshine of the face),강신성일,1973,505
20811,1833,21566,설원의 정,강신성일,1970,505
20812,1833,16194,에미 (Woman Requiem),강신성일,1985,505
20813,1833,21561,설녀,강신성일,1968,505
20814,1833,21911,아파트의 여인,강신성일,1969,505
20815,1833,93833,투 더 라스트 데이 (To the Last Day),강신성일,1960,505


In [14]:
# 배우별 출연한 영화수 확인
actors <- df[, c(1,4,6)]

# 중복 제거 
actors <- unique(actors)

# 출연 영화수 기준으로 내림차순 정렬
actors <- actors[order(actors$movieCnt, 
                       decreasing = TRUE),]

# 상위 20명만 확인
head(actors, 20L)

# 모두 옛날 배우들!! 
# 지금까지 활동 중인 배우는 '이순재, 박근형' 정도

,actorCode,actorName,movieCnt
20806,1833,강신성일,505
18565,1753,김지미,343
33247,29349,박노식,318
21790,18477,신영균,296
11810,15267,김진규,290
32239,2808,허장강,275
38564,3191,남궁원,264
67628,4264,윤정희,247
24948,2011,최무룡,240
61545,40872,김승호,228


In [15]:
# 총 배우수 확인 (26425명)
nrow(actors)

# 출연한 영화수가 10편 이상인 배우수만 확인 (2412명)
nrow(actors[actors$movieCnt >= 10,])

[1] 26425

[1] 2412

In [16]:
# 출연한 영화수가 10편 이상인 배우만 남기기
df1 <- df[df$movieCnt >= 10,]

# 중복을 포함한 데이터 프레임 행(영화) 개수 확인
nrow(df1)

[1] 64679

In [17]:
# 영화별 출연 배우수 컬럼 생성
numActor <- aggregate(x = df1$actorCode, 
                      by = list(df1$movieCode, df1$movieName), 
                      FUN = length)

# 컬럼명 바꾸기
colnames(numActor) <- c("movieCode","movieName","actorCnt")

# 첫 10줄만 확인
head(numActor, 10L)

movieCode,movieName,actorCnt
55758,...에 묻다 (...To Ask),1
33651,...한 후에 (Thereafter...),1
36957,...ing,6
131943,'CHE'CK,1
40028,@골뱅이,15
41338,007 폭소 대작전,5
29688,01412 파사신검 (01412 破邪神劍),3
145119,0시의 그녀,2
22301,0시의 부르스,3
22292,0시의 호텔 (Hotel At 00:00),16


In [18]:
# 전체 데이터에 병합
df1 <- merge(x = df1, 
             y = numActor[,c(1,3)], 
             by = "movieCode",
             all.x = TRUE)

# 영화별 배우수 빈도 확인
table(df1$actorCnt, useNA = "ifany")


   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
2506 3734 6567 6476 3805 3402 3717 4112 3483 3500 3443 2832 2405 2268 2055 1744 
  17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32 
1394 1224  969  600  546  550  345  264  200  364  162  168  116  150   62   96 
  33   34   35   36   38   41   42   44   45   46   48   49   50   54   55   65 
 198  102   70  108   76   82   42   44   45  138   48   49   50   54  110   65 
  68   71 
  68   71 

In [19]:
# 출연 배우수가 가장 많은 영화 확인
df1[df1$actorCnt == max(df1$actorCnt),]

# [주의] 출연한 영화수가 5편 이상인 배우들만 대상으로 했으므로 실제 데이터와 다름!!
# 실제로 가장 많은 배우가 출연한 영화는 "개그콘서트"임

,movieCode,actorCode,movieName,actorName,year,movieCnt,actorCnt
32773,23841,7234,용의 눈물,김성찬,1996,22,71
32774,23841,43606,용의 눈물,장성원,1996,12,71
32775,23841,6458,용의 눈물,남일우,1996,37,71
32776,23841,42434,용의 눈물,손호균,1996,18,71
32777,23841,7207,용의 눈물,강태기,1996,33,71
32778,23841,3179,용의 눈물,이민우,1996,41,71
32779,23841,48545,용의 눈물,안연홍,1996,31,71
32780,23841,40014,용의 눈물,신동훈,1996,13,71
32781,23841,7661,용의 눈물,이상인,1996,12,71
32782,23841,2966,용의 눈물,장인한,1996,36,71


In [20]:
# 출연한 배우수가 2명 이상인 영화만 남기기
df1 <- df1[df1$actorCnt >= 2,]

# 행 개수 확인
nrow(df1)

# 10편 이상 출연한 배우들이 출연한 영화 편수 (중복 제거)
length(unique(df1$movieCode))

[1] 62173

[1] 10316

In [21]:
# 컬럼 정렬
df1 <- df1[, c("movieCode","movieName","year","actorCnt","actorCode","actorName","movieCnt")]

# 첫 10줄만 확인
head(df1, 10L)

,movieCode,movieName,year,actorCnt,actorCode,actorName,movieCnt
1,100001,와라! 편의점,2009,2,97542,신용우,14
2,100001,와라! 편의점,2009,2,48841,시영준,12
3,100017,검지손가락,2011,2,48937,소이,16
4,100017,검지손가락,2011,2,5770,류덕환,34
7,100023,부곡 하와이 (Bugok Hawaii: Illusionary Paradise),2015,4,46662,박명신,38
8,100023,부곡 하와이 (Bugok Hawaii: Illusionary Paradise),2015,4,288291,류혜린,11
9,100023,부곡 하와이 (Bugok Hawaii: Illusionary Paradise),2015,4,53939,오성태,11
10,100023,부곡 하와이 (Bugok Hawaii: Illusionary Paradise),2015,4,39885,김세동,20
12,100030,피를 파는 사나이 (The Blood Seller),2012,2,159435,윤승훈,16
13,100030,피를 파는 사나이 (The Blood Seller),2012,2,149212,송삼동,29


In [22]:
# 2000년 이후 제작된 영화만 남기기
# 아주 옛날 영화까지 포함하면 누군지 모르니까... 
df2 <- df1[df1$year >= 2000,]

# 행 개수 확인
nrow(df2)

[1] 36100

In [23]:
# 영화-배우 행렬을 만들어 같은 영화에 동시 출연한 배우들의 횟수를 확인함 
# 이를 위해 df2 전처리 (영화코드와 배우코드만 남기고 각 행별 1씩 할당)
maDf <- cbind(df2[, c("movieCode","actorCode")],
              check = rep(1, nrow(df2)))

# 첫 10줄만 미리보기
head(maDf, 10L)

,movieCode,actorCode,check
1,100001,97542,1
2,100001,48841,1
3,100017,48937,1
4,100017,5770,1
7,100023,46662,1
8,100023,288291,1
9,100023,53939,1
10,100023,39885,1
12,100030,159435,1
13,100030,149212,1


In [24]:
# reshape2 패키지의 acast() 함수를 이용하여 영화-배우 행렬 생성
# 행렬의 빈 요소는 0으로 채움 
maMat <- acast(data = maDf, 
               formula = movieCode ~ actorCode, 
               fill = 0)

# 행렬 차원수 확인
dim(maMat)

# 10행 * 10열만 확인
# 대부분 0인 행렬이 생성됨 
maMat[1:10, 1:10]

Using check as value column: use value.var to override.
Aggregation function missing: defaulting to length


[1] 4960 2082

,100207,100854,100860,101096,102416,102521,102523,10319,10320,103201
100001,0,0,0,0,0,0,0,0,0,0
100017,0,0,0,0,0,0,0,0,0,0
100023,0,0,0,0,0,0,0,0,0,0
100030,0,0,0,0,0,0,0,0,0,0
100041,0,0,0,0,0,0,0,0,0,0
100165,0,0,0,0,0,0,0,0,0,0
100215,0,0,0,0,0,0,0,0,0,0
100306,0,0,0,0,0,0,0,0,0,0
100318,0,0,0,0,0,0,0,0,0,0
100354,0,0,0,0,0,0,0,0,0,0


In [25]:
# 배우 간 동시 출연 횟수를 행렬로 만들기
# 배우코드로 행과 열을 이루어야 하므로 영화-배우 행렬을 전치한 후 행렬 곱을 실행
actorsMul <- t(maMat) %*% maMat

# 같은 영화에 출연한 적이 있으면 1, 없으면 0을 할당
actorsMul <- ifelse(actorsMul >= 2, 1, 0)

# 같은 배우의 출연횟수(대각원소)는 0으로 치환함
diag(actorsMul) <- 0

# 10행 10열만 미리보기
actorsMul[1:10, 1:10]

,100207,100854,100860,101096,102416,102521,102523,10319,10320,103201
100207,0,0,0,0,0,0,0,0,0,0
100854,0,0,0,0,0,0,0,0,0,0
100860,0,0,0,0,0,0,0,0,0,0
101096,0,0,0,0,0,0,0,0,0,0
102416,0,0,0,0,0,0,0,1,0,0
102521,0,0,0,0,0,0,0,0,0,0
102523,0,0,0,0,0,0,0,0,0,0
10319,0,0,0,0,1,0,0,0,0,0
10320,0,0,0,0,0,0,0,0,0,0
103201,0,0,0,0,0,0,0,0,0,0


In [26]:
# 모든 행의 합이 0인 행 삭제하기 위해 위치 확인
zeros <- as.numeric(which(rowSums(actorsMul) == 0))

# 정방행렬을 유지하기 위해 열도 함께 삭제함
actorsMul <- actorsMul[-zeros, -zeros]

# 행렬의 차원수 확인
dim(actorsMul)

# 10행 10열만 미리보기
actorsMul[1:10, 1:10]

[1] 1913 1913

,100207,100854,100860,101096,102416,102521,10319,10320,103201,10338
100207,0,0,0,0,0,0,0,0,0,0
100854,0,0,0,0,0,0,0,0,0,0
100860,0,0,0,0,0,0,0,0,0,0
101096,0,0,0,0,0,0,0,0,0,0
102416,0,0,0,0,0,0,1,0,0,0
102521,0,0,0,0,0,0,0,0,0,0
10319,0,0,0,0,1,0,0,0,0,0
10320,0,0,0,0,0,0,0,0,0,1
103201,0,0,0,0,0,0,0,0,0,0
10338,0,0,0,0,0,0,0,1,0,0


## 최단거리 구하기 (케빈 베이컨 놀이)

In [27]:
# cell value가 1인 컬럼 번호 가져오기 
ones <- list()
for (i in 1:nrow(actorsMul)) {
  ones[[i]] <- as.numeric(which(actorsMul[,i] == 1))
}

In [28]:
# 행이름과 열이름으로 데이터프레임 생성하기
actorsDf <- data.frame()
for (i in 1:length(ones)) {
  coDf <- data.frame(from = rownames(actorsMul)[i],
                     to = colnames(actorsMul)[ones[[i]]])
  actorsDf <- rbind(actorsDf, coDf)
}

In [29]:
# 중복 제거 후 차원 확인 
actorsDf <- unique(actorsDf)
dim(actorsDf)

[1] 41444     2

In [30]:
# graph 객체를 생성
graphObj <- graph.data.frame(actorsDf, directed = FALSE)

# 각 노드(영화배우)별 최단경로 구하기
shortPath <- shortest.paths(graphObj)

# 범위 확인
# 최대값이 Inf인 이유는 함께 출연하지 않은 배우들이 있기 때문 
range(shortPath)

[1]   0 Inf

In [31]:
# 따라서 연결이 안되는 배우 간 경로는 99로 치환! 
# NA로 변경하면, 나중에 평균에서 제외되므로 오히려 무명의 배우가 가장 짧은 결과를 보이기 때문
shortPath <- ifelse(shortPath == Inf, 99, shortPath)
range(shortPath)

[1]  0 99

In [32]:
# 최단경로의 행 기준 평균 구하기
pathMean <- as.data.frame(rowMeans(shortPath))

# 최단경로의 평균 범위 확인
summary(pathMean)

# 배우코드 컬럼 생성 
pathMean$actorCode <- rownames(shortPath)

# 행번호는 초기화하고, 컬럼명 새로 지정
rownames(pathMean) <- c()
colnames(pathMean) <- c("pathMean","actorCode")

 rowMeans(shortPath)
 Min.   : 2.431     
 1st Qu.: 3.003     
 Median : 3.221     
 Mean   : 3.619     
 3rd Qu.: 3.497     
 Max.   :98.897     

In [33]:
# actors 객체에 병합한 후 정리하기 
actors1 <- merge(x = actors, 
                 y = pathMean, 
                 by = "actorCode", 
                 all.x = TRUE)

# NA인 행 제거 
actors1 <- actors1[is.na(actors1$pathMean) == FALSE,]

# 최단경로의 평균값이 작은 순서대로 오름차순 정렬
actors1 <- actors1[order(actors1$pathMean, decreasing = FALSE),]

In [34]:
# 평균이 가장 낮은 상위 20명 확인
head(actors1, 20L)

,actorCode,actorName,movieCnt,pathMean
25803,8775,이한위,99,2.431260
5264,1706,김갑수,78,2.451647
24098,7137,이순재,207,2.484056
13835,37945,박철민,83,2.501307
24247,7276,이대연,76,2.510716
95,10320,조희봉,64,2.512807
25248,8158,안내상,82,2.513853
6581,2338,안석환,75,2.514898
1566,12389,강신일,60,2.515944
21217,5546,이원종,65,2.516466


# End of Document